In [16]:
import os
import sqlite3
import pandas as pd
from pyprojroot import here
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain.tools import tool
from langchain.agents import create_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate


ImportError: cannot import name 'AgentExecutor' from 'langchain.agents' (/home/qul/Developments/Projects/Mine/llm-fun-project/.venv/lib/python3.12/site-packages/langchain/agents/__init__.py)

In [ ]:
db_dir = here("data/sql")
os.makedirs(db_dir, exist_ok=True)

db_path = here("data/sql/sales.db")

conn = sqlite3.connect(db_path)

data = {
    "customer_name": ["Andi", "Budi", "Citra", "Dewi", "Andi", "Budi"],
    "amount": [200, 150, 400, 250, 180, 300],
    "product": ["Laptop", "Keyboard", "Monitor", "Mouse", "SSD", "HDD"]
}

df = pd.DataFrame(data)

df.to_sql("sales", conn, index=False, if_exists="replace")

conn.close()

print(f"✅ Database '{db_path}' berhasil dibuat!")


✅ Database '/home/qul/Developments/Projects/Mine/llm-fun-project/data/sql/sales.db' berhasil dibuat!


In [12]:
import pandas as pd
import sqlite3
from typing import Dict
from langchain_core.tools import tool
from pyprojroot import here  

SQL_CACHE: Dict[str, pd.DataFrame] = {}

@tool
def query_and_cache(database_path: str, query: str) -> str:
    """
    Jalankan query SQL dan simpan hasilnya ke cache global agar tidak perlu diulang.
    Path database otomatis diselesaikan relatif terhadap root project.
    """
    import os
    from pathlib import Path

    db_path = here(database_path)
    db_path = str(db_path.resolve())

    if not Path(db_path).exists():
        return f"❌ Database tidak ditemukan di: {db_path}"

    cache_key = f"{db_path}:{query}"

    if cache_key in SQL_CACHE:
        return f"📦 Hasil query sudah ada di cache: {cache_key}"

    conn = sqlite3.connect(db_path)
    df = pd.read_sql(query, conn)
    conn.close()

    SQL_CACHE[cache_key] = df

    return f"✅ Query berhasil dijalankan dan disimpan di cache: {cache_key}"


@tool
def show_cached_result(database_path: str, query: str) -> str:
    """
    Menampilkan 5 baris pertama hasil query dari cache (kalau sudah ada).
    Path database otomatis diselesaikan relatif terhadap root project.
    """
    from pathlib import Path

    db_path = here(database_path)
    db_path = str(db_path.resolve())
    cache_key = f"{db_path}:{query}"

    if cache_key not in SQL_CACHE:
        return "⚠️ Data belum ada di cache. Jalankan query_and_cache dulu."
    
    df = SQL_CACHE[cache_key]
    return df.head().to_string(index=False)


In [ ]:
query = "SELECT customer_name, SUM(amount) as total_spent FROM sales GROUP BY customer_name"


print(query_and_cache.invoke({"database_path": "data/sql/sales.db", "query": query}))

# 🔹 Langkah 2: Coba tampilkan hasil dari cache
print(show_cached_result.invoke({"database_path": "data/sql/sales.db", "query": query}))

# 🔹 Langkah 3: Jalankan lagi query yang sama
print(query_and_cache.invoke({"database_path": "data/sql/sales.db", "query": query}))

# 🔹 Langkah 4: Query baru
query2 = "SELECT product, AVG(amount) as avg_price FROM sales GROUP BY product"
print(query_and_cache.invoke({"database_path": "data/sql/sales.db", "query": query2}))

# 🔹 Langkah 5: Lihat hasil query baru dari cache
print(show_cached_result.invoke({"database_path": "data/sql/sales.db", "query": query2}))


✅ Query berhasil dijalankan dan disimpan di cache: /home/qul/Developments/Projects/Mine/llm-fun-project/data/sql/sales.db:SELECT customer_name, SUM(amount) as total_spent FROM sales GROUP BY customer_name
customer_name  total_spent
         Andi          380
         Budi          450
        Citra          400
         Dewi          250
📦 Hasil query sudah ada di cache: /home/qul/Developments/Projects/Mine/llm-fun-project/data/sql/sales.db:SELECT customer_name, SUM(amount) as total_spent FROM sales GROUP BY customer_name
✅ Query berhasil dijalankan dan disimpan di cache: /home/qul/Developments/Projects/Mine/llm-fun-project/data/sql/sales.db:SELECT product, AVG(amount) as avg_price FROM sales GROUP BY product
 product  avg_price
     HDD      300.0
Keyboard      150.0
  Laptop      200.0
 Monitor      400.0
   Mouse      250.0
